In [18]:
import sqlite3
import pandas as pd
import numpy as np
import sys
import glob, os
from pyteomics import mgf

In [19]:
RT_LIMIT_LOWER = 0
RT_LIMIT_UPPER = 3000

In [20]:
RT_BUFFER_WIDTH = 0

In [21]:
BASE_NAME = '190719_Hela_Ecoli_1to1_01_Slot1-1_1_1926'

In [22]:
BASE_MAXQUANT_DIR = '/Users/darylwilding-mcbride/Downloads/{}-combined'.format(BASE_NAME)
MAXQUANT_TXT_DIR = '{}/txt'.format(BASE_MAXQUANT_DIR)
ALLPEPTIDES_FILENAME = '{}/allPeptides.txt'.format(MAXQUANT_TXT_DIR)
APL_DIR = '{}/andromeda'.format(BASE_MAXQUANT_DIR)

In [23]:
MGF_FILE_NAME = '{}/{}-mq.mgf'.format(BASE_MAXQUANT_DIR, BASE_NAME)

In [24]:
MIN_ISOTOPE_CORRELATION = 0.9

allpeptides_df = pd.read_csv(ALLPEPTIDES_FILENAME, sep='\t')
allpeptides_df.rename(columns={'Number of isotopic peaks':'isotope_count', 'm/z':'mz', 'Number of data points':'number_data_points', 'Intensity':'intensity', 'Ion mobility index':'scan', 'Ion mobility index length':'scan_length', 'Ion mobility index length (FWHM)':'scan_length_fwhm', 'Retention time':'rt', 'Retention length':'rt_length', 'Retention length (FWHM)':'rt_length_fwhm', 'Charge':'charge_state', 'Number of pasef MS/MS':'number_pasef_ms2_ids', 'Pasef MS/MS IDs':'pasef_msms_ids', 'MS/MS scan number':'msms_scan_number', 'Isotope correlation':'isotope_correlation'}, inplace=True)
allpeptides_df = allpeptides_df[allpeptides_df.intensity.notnull() & (allpeptides_df.number_pasef_ms2_ids > 0) & (allpeptides_df.msms_scan_number >= 0) & allpeptides_df.pasef_msms_ids.notnull() & (allpeptides_df.isotope_correlation >= MIN_ISOTOPE_CORRELATION) & (allpeptides_df.rt >= RT_LIMIT_LOWER+RT_BUFFER_WIDTH) & (allpeptides_df.rt <= RT_LIMIT_UPPER-RT_BUFFER_WIDTH)].copy()


In [25]:
allpeptides_df.msms_scan_number = allpeptides_df.msms_scan_number.apply(lambda x: int(x))

In [26]:
allpeptides_df.msms_scan_number.max()

39270

In [27]:
allpeptides_df.tail()

,Raw file,charge_state,mz,Mass,Resolution,number_data_points,Number of frames,isotope_count,isotope_correlation,Mass fractional part,...,Min frame index,Max frame index,scan,scan_length,scan_length_fwhm,intensity,Intensities,number_pasef_ms2_ids,pasef_msms_ids,msms_scan_number
191580,190719_Hela_Ecoli_1to1_01_Slot1-1_1_1926,2,580.28097,1158.54740,24593.888676,11088,33,3,0.958905,0.547387,...,343,375,891,54,27,174960.0,NaN,4,6625;6633;6641;6649,3881
191589,190719_Hela_Ecoli_1to1_01_Slot1-1_1_1926,3,712.94858,2135.82390,25287.251692,11715,22,5,0.993154,0.823906,...,186,207,897,45,18,141950.0,NaN,2,3575;3583,31608
191596,190719_Hela_Ecoli_1to1_01_Slot1-1_1_1926,2,512.25803,1022.50150,24976.754027,3429,29,3,0.997006,0.501507,...,170,198,903,21,12,51455.0,NaN,2,2828;2835,1033
191609,190719_Hela_Ecoli_1to1_01_Slot1-1_1_1926,2,472.28221,942.54987,25029.184983,19411,40,4,0.999323,0.549866,...,97,136,891,42,24,651280.0,NaN,1,1502,629
191629,190719_Hela_Ecoli_1to1_01_Slot1-1_1_1926,2,537.74573,1073.47690,25014.985918,8882,64,3,0.965886,0.476899,...,30,93,903,24,3,102050.0,NaN,4,800;804;808;812,1810


In [28]:
allpeptides_df.columns

Index(['Raw file', 'charge_state', 'mz', 'Mass', 'Resolution',
       'number_data_points', 'Number of frames', 'isotope_count',
       'isotope_correlation', 'Mass fractional part', 'Mass deficit', 'rt',
       'rt_length', 'rt_length_fwhm', 'Min frame index', 'Max frame index',
       'scan', 'scan_length', 'scan_length_fwhm', 'intensity', 'Intensities',
       'number_pasef_ms2_ids', 'pasef_msms_ids', 'msms_scan_number'],
      dtype='object')

In [29]:
def collate_spectra_for_feature(ms1_d, ms2_df):
    # append the monoisotopic and the ms2 fragments to the list for MGF creation
    pairs_df = ms2_df[['mz', 'intensity']].copy().sort_values(by=['intensity'], ascending=False)
    spectrum = {}
    spectrum["m/z array"] = pairs_df.mz.values
    spectrum["intensity array"] = pairs_df.intensity.values
    params = {}
    params["TITLE"] = "RawFile: {} Index: {} Charge: {} FeatureIntensity: {} RtApex: {}".format(ms1_d['raw_file'], ms1_d['mq_index'], ms1_d['charge'], ms1_d['intensity'], round(ms1_d['rt_apex'],2))
    params["INSTRUMENT"] = "ESI-QUAD-TOF"
    params["PEPMASS"] = "{} {}".format(round(ms1_d['monoisotopic_mz'],6), ms1_d['intensity'])
    params["CHARGE"] = "{}+".format(ms1_d['charge'])
    params["RTINSECONDS"] = "{}".format(round(ms1_d['rt_apex'],2))
    params["SCANS"] = "{}".format(int(ms1_d['rt_apex']))
    spectrum["params"] = params
    return spectrum

In [30]:
# build a list of indexes from the APL files
ms2_peaks = []
apl_indexes = []
for file in glob.glob("{}/*.apl".format(APL_DIR)):
    with open(file, 'r') as f:
        for line in f:
            line = line.rstrip()
            if len(line) > 0:
                if line.startswith("header="):
                    mq_index = int(line.split(' ')[3])
                if line[0].isdigit():
                    line_a = line.split('\t')
                    mz = float(line_a[0])
                    intensity = round(float(line_a[1]))
                    ms2_peaks.append((mz, intensity))
                if line.startswith("peaklist end"):
                    apl_indexes.append((mq_index, ms2_peaks.copy()))
                    del ms2_peaks[:]
                    mq_index = 0
apl_indexes_df = pd.DataFrame(apl_indexes, columns=['mq_index','ms2_peaks'])

In [31]:
apl_indexes_df.head()

,mq_index,ms2_peaks
0,11789,"[(211.14923, 42), (232.55944, 4), (233.13281, ..."
1,11790,"[(198.08868, 16), (199.18705, 10), (228.13997,..."
2,11791,"[(262.6408, 8), (328.90376, 4), (329.18094, 45..."
3,11792,"[(232.11246, 17), (233.08101, 8), (233.15044, ..."
4,11793,"[(225.10429, 13), (228.17787, 9), (229.1315, 4..."


In [32]:
mgf_spectra = []
for idx,row in allpeptides_df.iterrows():
    mq_index = row.msms_scan_number
    ms1_d = {'monoisotopic_mass':row.Mass, 
             'charge':row.charge_state, 
             'monoisotopic_mz':row.mz, 
             'intensity':int(row.intensity), 
             'scan_apex':row.scan, 
             'rt_apex':row.rt,
             'raw_file':row['Raw file'],
             'mq_index':mq_index}
    df = apl_indexes_df[apl_indexes_df.mq_index == mq_index]
    if len(df) == 1:
        ms2_peaks_df = pd.DataFrame(df.ms2_peaks.values[0], columns=['mz','intensity'])
        feature_spectra = collate_spectra_for_feature(ms1_d, ms2_peaks_df)
        mgf_spectra.append(feature_spectra)
    else:
        print("no match for mq_index {}".format(mq_index))

In [33]:
len(mgf_spectra)

36448

In [34]:
allpeptides_df.msms_scan_number.nunique()

36448

In [35]:
# generate the MGF for all the features
print("generating the MGF: {}".format(MGF_FILE_NAME))
if os.path.isfile(MGF_FILE_NAME):
    os.remove(MGF_FILE_NAME)
f = mgf.write(output=MGF_FILE_NAME, spectra=mgf_spectra)


generating the MGF: /Users/darylwilding-mcbride/Downloads/190719_Hela_Ecoli_1to1_01_Slot1-1_1_1926-combined/190719_Hela_Ecoli_1to1_01_Slot1-1_1_1926-mq.mgf
